In [1]:
!pip install -q bitsandbytes accelerate transformers datasets  peft tqdm evaluate scikit-learn huggingface_hub


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import transformers
print(transformers.__version__)

4.55.4


In [3]:
from huggingface_hub import notebook_login

In [5]:
notebook_login()

In [ ]:
#############################

In [6]:
import numpy as np, torch
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          BitsAndBytesConfig, Trainer, TrainingArguments)
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch.nn.functional as F

MODEL = "meta-llama/Llama-3.2-1B-Instruct"
USE_CLASS_WEIGHTS = False  # flip to True if your labels are imbalanced

In [7]:
# 1) Data
ds = load_dataset("ag_news")
train_val = ds["train"].train_test_split(test_size=0.1, seed=42, stratify_by_column="label")
train_ds, val_ds, test_ds = train_val["train"], train_val["test"], ds["test"]
num_labels = ds["train"].features["label"].num_classes  # 4

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [8]:
ds["train"]["text"][1234],ds["train"]["label"][1234]

('Yahoo to Sell Domain Names (AP) AP - Yahoo Inc. plans to start selling Internet domain names Tuesday as part of its expanding services for small businesses.',
 3)

In [9]:
print(ds["train"].features)

{'text': Value('string'), 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'])}


In [10]:
# 2) Tokenizer
tok = AutoTokenizer.from_pretrained(MODEL, use_fast=True)
tok.pad_token = tok.eos_token
tok.pad_token_id = tok.eos_token_id

def preprocess(ex):
    return tok(ex["text"], truncation=True, max_length=512)

def to_torch(dataset):
    d = dataset.map(preprocess, batched=True)
    d.set_format(type="torch", columns=["input_ids","attention_mask","label"])
    return d

train_t, val_t, test_t = to_torch(train_ds), to_torch(val_ds), to_torch(test_ds)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Map:   0%|          | 0/108000 [00:00<?, ? examples/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [11]:
# 3) Model + LoRA (8-bit base)
quant = BitsAndBytesConfig(load_in_8bit=True, bnb_8bit_compute_dtype=torch.float16)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL, num_labels=num_labels, quantization_config=quant, device_map="auto"
)
lora = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.1, bias="none", task_type="SEQ_CLS",
    target_modules=["q_proj","k_proj","v_proj","o_proj"], modules_to_save=["score"]
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora)
model.config.pad_token_id = tok.pad_token_id


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = torch.compile(model)

In [15]:
# 4) Optional class weights (only if imbalanced)
class_weights = None
if USE_CLASS_WEIGHTS:
    counts = np.bincount(train_t["label"].numpy(), minlength=num_labels)
    w = (counts.sum() / (num_labels * counts)).astype(np.float32)  # inverse-freq normalized
    class_weights = torch.tensor(w, dtype=torch.float32)

# 5) Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred.predictions, eval_pred.label_ids
    preds = np.argmax(logits, axis=1)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=0)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": prec, "recall": rec}

# 6) Trainer (override loss only if using weights)
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # if not using weights, fall back to default
        if not USE_CLASS_WEIGHTS:
            return super().compute_loss(model, inputs, return_outputs=return_outputs, **kwargs)

        # Hugging Face Trainer expects labels in inputs
        labels = inputs.pop("labels")

        # forward pass
        outputs = model(**inputs)
        logits = outputs.logits

        # weighted loss
        loss = F.cross_entropy(
            logits, 
            labels, 
            weight=class_weights.to(logits.device)
        )

        return (loss, outputs) if return_outputs else loss


args = TrainingArguments(
    output_dir="ag_news_lora",
    learning_rate=2e-4,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=1,
    num_train_epochs=1,               # bump to 3–5 for real runs
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
    # fp16=True,
    bf16=True, # set to True if using A100 or 3090+ with latest drivers
    gradient_checkpointing=False, # disable for speed unless memory is tight
    warmup_ratio=0.1
)

trainer = WeightedTrainer(
    model=model, args=args,
    train_dataset=train_t, eval_dataset=val_t,
    processing_class=tok, compute_metrics=compute_metrics
)

In [16]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.151400,0.154771,0.946750,0.946769,0.947147,0.946750


TrainOutput(global_step=1688, training_loss=0.18809310067886426, metrics={'train_runtime': 969.2925, 'train_samples_per_second': 111.421, 'train_steps_per_second': 1.741, 'total_flos': 7.671380556526387e+16, 'train_loss': 0.18809310067886426, 'epoch': 1.0})

In [17]:
testing = trainer.evaluate(test_t)

/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [18]:
print(testing)

{'eval_loss': 0.16194745898246765, 'eval_accuracy': 0.9446052631578947, 'eval_f1': 0.9445910892121969, 'eval_precision': 0.9448193793984099, 'eval_recall': 0.9446052631578947, 'eval_runtime': 17.4543, 'eval_samples_per_second': 435.424, 'eval_steps_per_second': 6.818, 'epoch': 1.0}


In [19]:
!pip install plotly

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 46.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [22]:
# pip install plotly
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

def smooth(x, window=5):
    if window <= 1 or len(x) < window:
        return np.array(x)
    w = np.ones(window) / window
    return np.convolve(x, w, mode="valid")

def plot_trainer_history_plotly(trainer, output_dir="ag_news_lora",
                                smoothing_window=1, save_html=True, show=True):
    """
    Interactive Plotly visualization for HuggingFace Trainer logs.
    - Training loss (per logged step) -> train_loss.html
    - Evaluation metrics (all eval_*) -> eval_metrics.html
    """
    os.makedirs(output_dir, exist_ok=True)
    logs = trainer.state.log_history  # list of dicts

    # --- Training loss (per step) ---
    train_entries = [l for l in logs if "loss" in l and "step" in l]
    if len(train_entries) > 0:
        steps = [int(l["step"]) for l in train_entries]
        losses = [float(l["loss"]) for l in train_entries]

        fig_loss = go.Figure()
        fig_loss.add_trace(go.Scatter(
            x=steps, y=losses, mode="lines+markers", name="loss (raw)",
            hovertemplate="step: %{x}<br>loss: %{y:.6f}<extra></extra>"
        ))

        if smoothing_window and smoothing_window > 1 and len(losses) >= smoothing_window:
            y_s = smooth(np.array(losses), smoothing_window)
            x_s = steps[(smoothing_window - 1):]  # align x for 'valid' conv
            fig_loss.add_trace(go.Scatter(
                x=x_s, y=y_s, mode="lines", name=f"smoothed (w={smoothing_window})",
                hovertemplate="step: %{x}<br>loss: %{y:.6f}<extra></extra>"
            ))

        fig_loss.update_layout(
            title="Training loss (per logged step)",
            xaxis_title="step",
            yaxis_title="loss",
            template="plotly_white",
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
        )

        if save_html:
            outp = os.path.join(output_dir, "train_loss.html")
            fig_loss.write_html(outp, include_plotlyjs="cdn")
            print(f"Saved training loss HTML -> {outp}")
        if show:
            fig_loss.show()
    else:
        print("No training-loss entries found in trainer.state.log_history.")

    # --- Evaluation metrics ---
    eval_entries = [l for l in logs if any(k.startswith("eval_") for k in l.keys())]
    if len(eval_entries) == 0:
        print("No evaluation entries found in trainer.state.log_history.")
        return

    # Collect metric names (strip "eval_" prefix)
    metric_names = set()
    for e in eval_entries:
        for k in e.keys():
            if k.startswith("eval_"):
                metric_names.add(k[len("eval_"):])

    fig_eval = go.Figure()
    any_epoch_values = any("epoch" in e for e in eval_entries)

    for m in sorted(metric_names):
        xs, ys = [], []
        for e in eval_entries:
            val = e.get(f"eval_{m}", None)
            if val is None:
                continue
            # Prefer epoch for x-axis if present; fallback to step; else create indices later
            x = e.get("epoch", e.get("step", None))
            xs.append(float(x) if x is not None else None)
            ys.append(float(val))
        if len(xs) == 0:
            continue
        # If some x are None, replace with 1..N sequence
        if any(x is None for x in xs):
            xs = list(range(1, len(ys) + 1))
        fig_eval.add_trace(go.Scatter(
            x=xs, y=ys, mode="lines+markers", name=m,
            hovertemplate="step: %{x}<br>loss: %{y:.6f}<extra></extra>"
        ))

    xaxis_label = "epoch" if any_epoch_values else "eval step / index"
    fig_eval.update_layout(
        title="Evaluation metrics",
        xaxis_title=xaxis_label,
        yaxis_title="metric value",
        template="plotly_white",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )

    if save_html:
        outp = os.path.join(output_dir, "eval_metrics.html")
        fig_eval.write_html(outp, include_plotlyjs="cdn")
        print(f"Saved evaluation metrics HTML -> {outp}")
    if show:
        fig_eval.show()

# Example usage (call after trainer.train()):
plot_trainer_history_plotly(trainer, output_dir=args.output_dir, smoothing_window=5, save_html=True, show=True)


Saved training loss HTML -> ag_news_lora/train_loss.html


Saved evaluation metrics HTML -> ag_news_lora/eval_metrics.html
